In [ ]:
import numpy as np
import pandas as pd
import datetime
import ast
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
# Dataset downloaded from: https://www.kaggle.com/datasets/virajraut25/tmdb-6000-dataset

In [ ]:
movies = pd.read_csv('/content/tmdb_6000_movies.csv')
credits = pd.read_csv('/content/tmdb_6000_credits.csv')

In [ ]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [ ]:
movies.shape

(5798, 20)

In [ ]:
credits.head(2)

,id,cast,crew
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
movies = movies.merge(credits,on='id')

In [ ]:
movies.isnull().sum()

budget                     0
genres                     0
homepage                3417
id                         0
keywords                   0
original_language          0
original_title             0
overview                   7
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                 1072
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
dtype: int64

In [ ]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [ ]:
movies.head(2)
#genres
#id
#keywords
#overview
#production_companies
#release_date
#runtime
#title
#vote_average
#cast
#crew

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
movies = movies[['id','title','overview','genres','keywords','production_companies','cast','crew','runtime','release_date','vote_average']]

In [ ]:
movies.head(2)

,id,title,overview,genres,keywords,production_companies,cast,crew,runtime,release_date,vote_average
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",162.0,2009-12-10,7.2
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",169.0,2007-05-19,6.9
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",148.0,2015-10-26,6.3
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",165.0,2012-07-16,7.6
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",132.0,2012-03-07,6.1


In [ ]:
movies['release_date'] = pd.to_datetime(movies['release_date'])
movies['year'] = movies['release_date'].dt.year
movies.drop(columns=['release_date'], inplace=True)

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.shape

(5790, 11)

In [ ]:
movies.isnull().sum()

id                      0
title                   0
overview                0
genres                  0
keywords                0
production_companies    0
cast                    0
crew                    0
runtime                 0
vote_average            0
year                    0
dtype: int64

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies.head(2)

,id,title,overview,genres,keywords,production_companies,cast,crew,runtime,vote_average,year
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{'id': 1463, 'name': 'culture clash'}, {'id':...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",162.0,7.2,2009.0
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",169.0,6.9,2007.0
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",148.0,6.3,2015.0
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{'id': 849, 'name': 'dc comics'}, {'id': 853,...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",165.0,7.6,2012.0
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",132.0,6.1,2012.0


In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head(2)

,id,title,overview,genres,keywords,production_companies,cast,crew,runtime,vote_average,year
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",162.0,7.2,2009.0
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",169.0,6.9,2007.0
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",148.0,6.3,2015.0
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",165.0,7.6,2012.0
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",132.0,6.1,2012.0


In [ ]:
movies['cast'] = movies['cast'].apply(convert)
movies.head(2)

,id,title,overview,genres,keywords,production_companies,cast,crew,runtime,vote_average,year
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",162.0,7.2,2009.0
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",169.0,6.9,2007.0
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",148.0,6.3,2015.0
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",165.0,7.6,2012.0
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",132.0,6.1,2012.0


In [ ]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
#movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.sample(2)

,id,title,overview,genres,keywords,production_companies,cast,crew,runtime,vote_average,year
3047,87093,Big Eyes,"The story of the awakening of the painter, Mar...",[Drama],"[wife husband relationship, artist, court case]","[{""name"": ""The Weinstein Company"", ""id"": 308},...","[Amy Adams, Christoph Waltz, Krysten Ritter]",[Tim Burton],105.0,6.800,2014.0
4157,309919,The Curse of Downers Grove,The town of Downers Grove looks like your aver...,"[Thriller, Mystery, Horror, Drama]","[self-defense, car mechanic, graduation, high ...","[{""name"": ""Myriad Pictures"", ""id"": 738}, {""nam...","[Bella Heathcote, Lucas Till, Helen Slater]",[Derick Martini],89.0,4.400,2015.0
4921,440471,Escape Plan 2: Hades,Ray Breslin manages an elite team of security ...,"[Action, Thriller, Mystery]","[artificial intelligence (a.i.), prison, marti...","[{'id': 90968, 'logo_path': None, 'name': 'Leo...","[Sylvester Stallone, Dave Bautista, Jesse Metc...",[Steven C. Miller],96.0,5.163,2018.0
2297,203833,The Book Thief,While subjected to the horrors of WWII Germany...,[Drama],"[world war ii, book, nazi germany, jewish]","[{""name"": ""Studio Babelsberg"", ""id"": 264}, {""n...","[Geoffrey Rush, Sophie Nélisse, Emily Watson]",[Brian Percival],131.0,7.400,2013.0
4567,15624,Conversations with Other Women,"Reunited at a wedding after many years, former...","[Drama, Romance]","[dialogue, talking, bridesmaid, wedding recept...","[{""name"": ""Prophecy Pictures Ltd."", ""id"": 4104...","[Helena Bonham Carter, Aaron Eckhart, Yury Tsy...",[Hans Canosa],84.0,6.200,2006.0


In [ ]:
movies['production_companies'] = movies['production_companies'].apply(convert)
movies.head(2)

,id,title,overview,genres,keywords,production_companies,cast,crew,runtime,vote_average,year
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],162.0,7.2,2009.0
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],169.0,6.9,2007.0
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],148.0,6.3,2015.0
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Legendary Pictures, Warner Bros., DC Entertai...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],165.0,7.6,2012.0
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",[Walt Disney Pictures],"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],132.0,6.1,2012.0


In [ ]:
movies['year'] = movies['year'].round().astype(int)
movies['runtime'] = movies['runtime'].round().astype(int)
movies['vote_average'] = movies['vote_average'].round().astype(int)
movies.head(2)

,id,title,overview,genres,keywords,production_companies,cast,crew,runtime,vote_average,year
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],162,7,2009
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],169,7,2007
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],148,6,2015
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Legendary Pictures, Warner Bros., DC Entertai...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],165,8,2012
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",[Walt Disney Pictures],"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],132,6,2012


In [ ]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)
movies['production_companies'] = movies['production_companies'].apply(collapse)

In [ ]:
movies.head()

,id,title,overview,genres,keywords,production_companies,cast,crew,runtime,vote_average,year
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[IngeniousFilmPartners, TwentiethCenturyFoxFil...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],162,7,2009
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[WaltDisneyPictures, JerryBruckheimerFilms, Se...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],169,7,2007
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[ColumbiaPictures, Danjaq, B24]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],148,6,2015
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[LegendaryPictures, WarnerBros., DCEntertainme...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],165,8,2012
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",[WaltDisneyPictures],"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],132,6,2012


In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head(2)

In [ ]:
# Convert numerical columns to lists of strings before concatenation
movies['year'] = movies['year'].apply(lambda x: [str(x)])
movies['runtime'] = movies['runtime'].apply(lambda x: [str(x)])
movies['vote_average'] = movies['vote_average'].apply(lambda x: [str(x)])

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] + movies['production_companies'] + movies['year'] + movies['runtime'] + movies['vote_average']

In [ ]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew', 'production_companies', 'year', 'runtime', 'vote_average'])
new.head(2)

In [ ]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head(2)

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [ ]:
new['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron IngeniousFilmPartners TwentiethCenturyFoxFilmCorporation DuneEntertainment LightstormEntertainment 2009 162 7'

In [ ]:
# Clean data: lemmatize and remove stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
punctuations = string.punctuation

In [ ]:
def preprocess(text):
    # Remove punctuation and convert to lowercase
    text = ''.join([char.lower() for char in text if char not in punctuations])
    return [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]

In [ ]:
# Apply preprocessing to tags
new['tags'] = new['tags'].apply(preprocess)  # Apply preprocessing
new['tags'] = new['tags'].apply(lambda x: " ".join(x))  # Convert list of words back to a single string

In [ ]:
new.tags[0]

'22nd century paraplegic marine dispatched moon pandora unique mission becomes torn following order protecting alien civilization action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron ingeniousfilmpartners twentiethcenturyfoxfilmcorporation duneentertainment lightstormentertainment 2009 162 7'

In [ ]:

# Vectorize tags using TF-IDF
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
vector = tfidf.fit_transform(new['tags']).toarray()

In [ ]:
vector.shape

(5790, 5000)

In [ ]:
# Compute cosine similarity
similarity = cosine_similarity(vector)

In [ ]:
similarity

array([[1.        , 0.01879486, 0.02616843, ..., 0.0057042 , 0.        ,
        0.06161853],
       [0.01879486, 1.        , 0.01115222, ..., 0.00612116, 0.01055076,
        0.01402656],
       [0.02616843, 0.01115222, 1.        , ..., 0.00627383, 0.04045916,
        0.03553622],
       ...,
       [0.0057042 , 0.00612116, 0.00627383, ..., 1.        , 0.03258675,
        0.03297141],
       [0.        , 0.01055076, 0.04045916, ..., 0.03258675, 1.        ,
        0.04449272],
       [0.06161853, 0.01402656, 0.03553622, ..., 0.03297141, 0.04449272,
        1.        ]])

In [ ]:
new[new['title'] == 'The Lego Movie'].index[0]

744

In [ ]:
# Recommendation function
def recommend(movie):
    index = new[new['title'].str.lower() == movie.lower()].index
    if index.empty:
        print(f"Movie '{movie}' not found in the dataset.")
        return

    distances = sorted(list(enumerate(similarity[index[0]])), reverse=True, key=lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [ ]:
recommend('Iron man')

Iron Man 2
Iron Man 3
Avengers: Age of Ultron
Ant-Man
Captain America: Civil War


In [ ]:
import pickle

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))